In [1]:
from keras.models import Model 
from keras.layers import Input, BatchNormalization, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Conv1D, UpSampling1D, Conv1DTranspose, Reshape
import np_utils
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
import os
from sklearn.preprocessing import StandardScaler
import os
import datetime
from sklearn.model_selection import train_test_split

In [2]:
poor = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_outputs_poor.csv", header=None)
middle = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_outputs_middle.csv", header=None)
rich = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_outputs_rich.csv", header=None)
sc = StandardScaler()
bank_reserves = []
for i in np.arange(0, poor.shape[0]):
    sample = pd.concat([poor.iloc[i], middle.iloc[i]], axis=0).T
    sample = pd.concat([sample, rich.iloc[i]], axis=0).T
    sample_std = sc.fit_transform(sample.to_frame())
    bank_reserves.append(sample_std)

In [3]:
bank_reserves = [sample.flatten() for sample in bank_reserves]
bank_reserves = np.asarray(bank_reserves)
bank_reserves_train, bank_reserves_test = train_test_split(bank_reserves, test_size=0.20, random_state=42)

In [4]:
bank_reserves_train = bank_reserves_train.reshape(bank_reserves_train.shape[0], 303, 1)
bank_reserves_test = bank_reserves_test.reshape(bank_reserves_test.shape[0], 303, 1)

In [5]:
bank_reserves.shape

(100000, 303)

In [6]:
input = Input(shape=(303,1))

encoded = Conv1D(filters=32, kernel_size=10, activation='relu', padding='same')(input)
encoded = Conv1D(filters=64, kernel_size=10, activation='relu', padding='same')(encoded)
encoded = Conv1D(filters=128, kernel_size=10, activation='relu', padding='same')(encoded)
encoded = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(encoded)

#encoded = Flatten()(encoded)

encoded = Dense(384, activation='relu')(encoded)
decoded = Dense(5, activation='relu')(encoded)
decoded = Dense(384, activation='relu')(decoded)

#decoded = Reshape((303, 1))(decoded)

decoded = Conv1DTranspose(filters=128, kernel_size=3, activation='relu', padding='same')(decoded)
decoded = Conv1DTranspose(filters=128, kernel_size=10, activation='relu', padding='same')(decoded)
decoded = Conv1DTranspose(filters=64, kernel_size=10, activation='relu', padding='same')(decoded)
decoded = Conv1DTranspose(filters=32, kernel_size=10, activation='relu', padding='same')(decoded)
decoded = Conv1DTranspose(filters=1, kernel_size=10, activation=None, padding='same')(decoded)

autoencoder = Model(input, decoded)
autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 303, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 303, 32)        │           352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 303, 64)        │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 303, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 303, 128)       │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 303, 384)       │        49,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 303, 5)         │         1,925 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 303, 384)       │         2,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose                │ (None, 303, 128)       │       147,584 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_1              │ (None, 303, 128)       │       163,968 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_2              │ (None, 303, 64)        │        81,984 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_3              │ (None, 303, 32)        │        20,512 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_4              │ (None, 303, 1)         │           321 │
│ (Conv1DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 620,358 (2.37 MB)

 Trainable params: 620,358 (2.37 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [ ]:
autoencoder.fit(bank_reserves_train, bank_reserves_train,
 epochs=20,
 batch_size=64,
 shuffle=True,
 validation_data=(bank_reserves_test, bank_reserves_test),
 verbose=1)

Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 346s 275ms/step - loss: 0.0740 - val_loss: 9.1406e-05
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 338s 271ms/step - loss: 5.1902e-04 - val_loss: 3.2661e-04
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 339s 271ms/step - loss: 4.7140e-04 - val_loss: 9.6340e-04
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 341s 273ms/step - loss: 3.9175e-04 - val_loss: 8.5190e-04
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 339s 271ms/step - loss: 3.2256e-04 - val_loss: 1.9950e-04
Epoch 6/20
 812/1250 ━━━━━━━━━━━━━━━━━━━━ 1:53 259ms/step - loss: 2.9742e-04